The idea is that we sample existing queries and sample entities and relation with similar characteristics in the dataset that is not found in training set.

In [1]:
import pandas as pd
import pathlib
from inductive_query.utils import UnseenQueryExtractor,CompletelyUnseenQueryExtractor
import os
import json, json5
data_path = '/data/PlanRGCN_ent_rel_relFreq'

In [2]:

dataset_path='/data/wikidata_3_class_full'
#ext = UnseenQueryExtractor(dataset_path)
com = CompletelyUnseenQueryExtractor(dataset_path)

In [3]:

path = '/data/wikidata_3_class_full'
rel_freq_p = '/data/metaKGStat/wikidata'
split_path = f"{path}/test_sampled.tsv"
name_dict = {
    0: '0s to 1s',
    1: '1s to 10s',
    2: '$>$ 10s',
}

In [4]:
def load_freq_dict(metaKG_dir_path, sort_descending = False):
    pred_freq_path = os.path.join(metaKG_dir_path, 'predicate/pred_stat/batches_response_stats/freq')
    pred_freq = {}
    freq_pred = {}
    for i in os.listdir(pred_freq_path):
        with open(os.path.join(pred_freq_path, i),'r') as f:
            try:
                data = json.load(f)
            except:
                try:
                    data = json5.load(f)
                except:
                    continue
            data = data['results']['bindings']
            for v in data:
                pred_freq[v['p1']['value']] = int(v['triples']['value'])
                try:
                    freq_pred[int(v['triples']['value'])].append(v['p1']['value'])
                except:
                    freq_pred[int(v['triples']['value'])] = [v['p1']['value']]
    
    if sort_descending:
        pred_freq_keys = sorted(pred_freq.keys(), key=lambda x: pred_freq[x], reverse=True)
        pred_freq2 = {}
        for i in pred_freq_keys:
            pred_freq2[i] = pred_freq[i]
        pred_freq = pred_freq2
    return pred_freq

def remove_rels_freq_dict(freq_dict: dict, rels: list):
    for r in rels:
        try:
            del freq_dict[r]
        except KeyError:
            continue
    return freq_dict

In [5]:
rel_freq = load_freq_dict(rel_freq_p,sort_descending = True)

In [6]:
def load_ent_pred_ext(path):
    ext = UnseenQueryExtractor(path)
    ext.set_train_pred_ents()
    ext.set_train_pred_queryIDs()
    ext.set_train_ent_queryIDs()
    ext.set_val_pred_ents()
    ext.set_val_pred_queryIDs()
    ext.set_val_ent_queryIDs()
    
    ext.set_test_pred_ents()
    ext.set_test_pred_queryIDs()
    ext.set_test_ent_queryIDs()
    return ext
    
ext = load_ent_pred_ext(path)

In [7]:

def save_train_ents_rels(ext, path,  rel_freq,dir_name='wikidata'):
    ext.train_ents
    sorted_train_ents = sorted(ext.train_ents.keys(), key=lambda x: len(ext.train_ents[x]), reverse=True)
    sorted_train_preds = sorted(ext.train_preds.keys(), key=lambda x: len(ext.train_preds[x]), reverse=True)
    
    sorted_val_ents = sorted(ext.val_ents.keys(), key=lambda x: len(ext.val_ents[x]), reverse=True)
    sorted_val_preds = sorted(ext.val_preds.keys(), key=lambda x: len(ext.val_preds[x]), reverse=True)

    rel_freq = remove_rels_freq_dict(rel_freq, sorted_train_preds)
    rel_freq = remove_rels_freq_dict(rel_freq, sorted_val_preds)
    
    dir_path = os.path.join(path, dir_name)
    os.makedirs(dir_path, exist_ok=True)
    for p in ['train_ents.json','train_preds.json', 'val_ents.json','val_preds.json', 'rels_sorted.json']:
        f_p = os.path.join(dir_path, p)
        with open(f_p, 'w') as ffp:
            if 'train' in p:
                if 'ents' in p:
                    json.dump(sorted_train_ents, ffp)
                elif 'pred' in p:
                    json.dump(sorted_train_preds, ffp)
            elif 'val' in p:
                if 'ents' in p:
                    json.dump(sorted_val_ents, ffp)
                    print(f_p)
                elif 'pred' in p:
                    json.dump(sorted_val_preds, ffp)
            elif 'rels_sorted' in p:
                json.dump(rel_freq, ffp)
    

In [8]:
save_train_ents_rels(ext, data_path,rel_freq, dir_name='wikidata')

/data/PlanRGCN_ent_rel_relFreq/wikidata/val_ents.json


In [9]:
from inductive_query.utils import UnseenQueryExtractor,CompletelyUnseenQueryExtractor
dataset_path='/data/DBpedia_3_class_full'
rel_freq_p = '/data/metaKGStat/dbpedia'
#ext = UnseenQueryExtractor(dataset_path)
com = CompletelyUnseenQueryExtractor(dataset_path)

In [10]:
import pathlib
path = '/data/DBpedia_3_class_full'
split_path = f"{path}/test_sampled.tsv"
name_dict = {
    0: '0s to 1s',
    1: '1s to 10s',
    2: '$>$ 10s',
}
ext = load_ent_pred_ext(path)
rel_freq = load_freq_dict(rel_freq_p,sort_descending = True)

In [11]:
save_train_ents_rels(ext, data_path, rel_freq, dir_name='DBpedia')

/data/PlanRGCN_ent_rel_relFreq/DBpedia/val_ents.json
